In [187]:
# Connect to google drive to fetch preprocessed data of MaskedFaceNet Dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [188]:
import numpy as np
import cv2
import gc
from sklearn.model_selection import KFold
from google.colab.patches import cv2_imshow

In [189]:
# Load Correctly Masked Faces
correctly_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset1/Correct/correct_features.npz")
# Load Incorrectly Masked Faces
incorrect_features=np.load("/content/drive/MyDrive/MSProjectData/Dataset1/Incorrect/incorrect_features.npz")
# Load Incorrectly Masked Labels
incorrect_labels=np.load("/content/drive/MyDrive/MSProjectData/Dataset1/Incorrect/incorrect_labels.npz")

In [190]:
# Extract Correctly Masked Features from npz 
correctly_masked=correctly_features['arr_0']
correctly_masked=correctly_masked/255
# Extract Incorrectly Masked Features from npz
incorrectly_masked=incorrect_features['arr_0']
incorrectly_masked=incorrectly_masked/255
# Extract Incorrectly Masked Labesl from npz
incorrectly_masked_labels=incorrect_labels['arr_0']

In [191]:
del correctly_features
del incorrect_features
del incorrect_labels

In [192]:
print(correctly_masked.shape,incorrectly_masked.shape)

(67193, 16, 16, 1) (67857, 16, 16, 1)


In [193]:
mouthChinArr=[]
maskChinArr=[]
noseMouthArr=[]
for i in range(len(incorrectly_masked)):
  # Divide Incorrectly Masked Faces in categories
  imageNameArr=incorrectly_masked_labels[i].split('/')[-1].split('.')[0].split('_')[1:]
  labelName="_".join([element for element in imageNameArr])
  if labelName=="Mask_Mouth_Chin":
    mouthChinArr.append(incorrectly_masked[i])
  elif labelName=="Mask_Chin":
    maskChinArr.append(incorrectly_masked[i])
  elif labelName=="Mask_Nose_Mouth":
    noseMouthArr.append(incorrectly_masked[i])
  else:
    print("Something went wrong")
mouthChinArr=np.asarray(mouthChinArr)
maskChinArr=np.asarray(maskChinArr)
noseMouthArr=np.asarray(noseMouthArr)

In [194]:
print(len(mouthChinArr),len(maskChinArr),len(noseMouthArr))

55626 6056 6175


In [195]:
# Preparing Training And Testing Data for 5-fold cross validation
trainingMouthChinArr=[]
testingMouthChinArr=[]
trainingMaskChinArr=[]
testingMaskChinArr=[]
trainingNoseMouthArr=[]
testingNoseMouthArr=[]
correctTrainingFeatures=[]
correctTestingFeatures=[]
kfold=KFold(n_splits=5,shuffle=True)
for train,test in kfold.split(range(len(mouthChinArr))):
  trainingMouthChinArr.append(mouthChinArr[train])
  testingMouthChinArr.append(mouthChinArr[test])
for train,test in kfold.split(range(len(maskChinArr))):
  trainingMaskChinArr.append(maskChinArr[train])
  testingMaskChinArr.append(maskChinArr[test])
for train,test in kfold.split(range(len(noseMouthArr))):
  trainingNoseMouthArr.append(noseMouthArr[train])
  testingNoseMouthArr.append(noseMouthArr[test])
for train,test in kfold.split(range(len(correctly_masked))):
  correctTrainingFeatures.append(correctly_masked[train])
  correctTestingFeatures.append(correctly_masked[test])
trainingMouthChinArr=np.asarray(trainingMouthChinArr)
testingMouthChinArr=np.asarray(testingMouthChinArr)
trainingMaskChinArr=np.asarray(trainingMaskChinArr)
testingMaskChinArr=np.asarray(testingMaskChinArr)
trainingNoseMouthArr=np.asarray(trainingNoseMouthArr)
testingNoseMouthArr=np.asarray(testingNoseMouthArr)
correctTrainingFeatures=np.asarray(correctTrainingFeatures)
correctTestingFeatures=np.asarray(correctTestingFeatures)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [196]:
print(trainingMouthChinArr.shape,testingMouthChinArr.shape,trainingMaskChinArr.shape,testingMaskChinArr.shape,trainingNoseMouthArr.shape,testingNoseMouthArr.shape,correctTrainingFeatures.shape,
      correctTestingFeatures.shape)

(5,) (5,) (5,) (5,) (5, 4940, 16, 16, 1) (5, 1235, 16, 16, 1) (5,) (5,)


In [197]:
# Import Tesortflow and Keras libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Conv2D,Flatten,MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping

In [198]:
accuracy=[]
correctAccuracy=[]
testingMaskChinArrAccuracy=[]
testingMouthChinArrAccuracy=[]
testingNoseMouthArrAccuracy=[]
for i in range(0,5):
  # Preparing Training And Testing Set
  x_train=np.concatenate((correctTrainingFeatures[i],trainingMaskChinArr[i],trainingMouthChinArr[i],trainingNoseMouthArr[i]),axis=0)
  x_test=np.concatenate((correctTestingFeatures[i],testingMaskChinArr[i],testingMouthChinArr[i],testingNoseMouthArr[i]),axis=0)
  y_train=np.concatenate((np.zeros(len(correctTrainingFeatures[i])),np.ones(len(trainingMaskChinArr[i])),np.ones(len(trainingMouthChinArr[i])),np.ones(len(trainingNoseMouthArr[i]))),axis=0)
  y_test=np.concatenate((np.zeros(len(correctTestingFeatures[i])),np.ones(len(testingMaskChinArr[i])),np.ones(len(testingMouthChinArr[i])),np.ones(len(testingNoseMouthArr[i]))),axis=0)
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  # CNN Model
  model=Sequential()
  model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=x_train[0].shape))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=32,activation='relu'))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  model.summary()
  class MyClass(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      # Stop the model if accuracy reaches 99.5%
      if logs.get('accuracy') > 0.995:
        self.model.stop_training=True
  model.fit(x_train,y_train,epochs=500,batch_size=128,shuffle=True,callbacks=[MyClass()])
  accuracy.append(model.evaluate(x_test,y_test)[1])
  correctAccuracy.append(model.evaluate(correctTestingFeatures[i],np.zeros(len(correctTestingFeatures[i])))[1])
  testingMaskChinArrAccuracy.append(model.evaluate(testingMaskChinArr[i],np.ones(len(testingMaskChinArr[i])))[1])
  testingMouthChinArrAccuracy.append(model.evaluate(testingMouthChinArr[i],np.ones(len(testingMouthChinArr[i])))[1])
  testingNoseMouthArrAccuracy.append(model.evaluate(testingNoseMouthArr[i],np.ones(len(testingNoseMouthArr[i])))[1])


(108038, 16, 16, 1) (108038,) (27012, 16, 16, 1) (27012,)
Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_98 (Conv2D)           (None, 12, 12, 128)       3328      
_________________________________________________________________
max_pooling2d_98 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 4, 4, 32)          36896     
_________________________________________________________________
max_pooling2d_99 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_49 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_147 (Dense)            (None, 128)               16512     
_____________________________________________________________

In [202]:
print("Average Accuracy:",sum(accuracy)/len(accuracy)*100)
print("\n")
print("Average Accuracy for Correctly Masked Faces:",sum(correctAccuracy)/len(correctAccuracy)*100)
print("Average Accuract for Incorrectly Masked only Chin Covered:",sum(testingMaskChinArrAccuracy)/len(testingMaskChinArrAccuracy)*100)
print("Average Accuract for Incorrectly Masked Mouth and Chin Covered:",sum(testingMouthChinArrAccuracy)/len(testingMouthChinArrAccuracy)*100)
print("Average Accuract for Incorrectly Masked Nose and Mouth Covered:",sum(testingNoseMouthArrAccuracy)/len(testingNoseMouthArrAccuracy)*100)

Average Accuracy: 97.90892362594604


Average Accuracy for Correctly Masked Faces: 97.81970143318176
Average Accuract for Incorrectly Masked only Chin Covered: 98.48088264465332
Average Accuract for Incorrectly Masked Mouth and Chin Covered: 99.09394860267639
Average Accuract for Incorrectly Masked Nose and Mouth Covered: 87.6437246799469


In [203]:
model.save("/content/drive/MyDrive/MSProjectData/Dataset1/MaskedFaceNetModel.h5")